# Lesson 2 - Session 2 - String mainpulation 

### Learning Objectives

1. Learn how to manipulate strings in Bash
2. Learn what is regular expression and how to use pattern matching it in Bash
3. Learn about useful tools like `jq`, `awk`, `sed` and `grep` for string manipulation in Bash
4. Show the example of string manipulation in Bash
5. Aknowledge the importance of string manipulation in data processing using bash script

## 1. Introduction

While bash script seems awkward for calculation and operations, it is very powerful for string manipulation. In this lesson, we will learn how to manipulate strings in Bash. We will also learn about regular expressions and how to use pattern matching in Bash. We will also learn about useful tools like `jq`, `awk`, `sed` and `grep` for string manipulation in Bash. We will show examples of string manipulation in Bash and acknowledge the importance of string manipulation in data processing using bash script.

### 1-1. Why bash script for string manipulation?

While it is true that every programming language can manipulate text, some languages are particularly well-suited for it. Python, for example, is a widely used tool known for its powerful built-in regular expression capabilities and data processing features. Perl is also one of the most powerful languages for string manipulation. However, learning Bash scripting for text processing is important as well. Not only is it the most commonly used language for system administration and automation, but it is also available on every Unix-like system by default, without requiring additional installation.

Python, however, is notorious for its poor package management. While it can be used both interactively in a console and as a script, installing Python on every system is not always straightforward. For instance, if you need to run a script on a stateless CI server like GitHub Actions, you must install Python 3 and its dependencies on every run. This increases build time due to the overhead of installing Python and its packages, even if you use modern tools like uv instead of pip for package management.

In contrast, Bash is available on every Unix-like system without any additional setup. You can execute Bash scripts on any Unix-like system immediately, without installation. This is why Bash remains the most commonly used language for system administration and automation.